In [1]:
import torch
import numpy as np
import scipy.io as sio
import torch.nn.functional as F
import os
import random
from sklearn.neighbors import kneighbors_graph
from torch.nn.functional import one_hot

def constructW(X, k, sigma):
    from sklearn.preprocessing import MinMaxScaler
    n = X.shape[1]

    W = (kneighbors_graph(X.T.cpu(), k+1, mode = 'distance', include_self = False, metric = 'euclidean')).toarray()
    W[W == 0] = np.infty

    W = np.exp((-(W**2))/(sigma**2))
    W = np.maximum(W, W.T)
    return torch.tensor(W).type(dtype)


def sim_dissim(y, n, p):
    l = np.fix(n*p).astype(np.uint8)
    numbers = random.sample(range(n), l)
    S = torch.zeros([n, n]).type(dtype)
    D = torch.zeros([n, n]).type(dtype)
    for i in numbers:
        for j in numbers:
            if y[i] == y[j]:
                S[i, j] = 1
            else:
                D[i, j] = 1
    DS = torch.diag(torch.sum(S, dim=0))
    return D, S, DS


def MakeLabel(data, per, c, n):
    arr = np.arange(n)
    np.random.shuffle(arr)
    data = data[arr, :]

    data = data[data[:, -1].sort()[1]]

    DATA = []
    nc = np.zeros(c).astype(int)
    for i in range(c):
        DATA.append(data[data[:, -1] == i])
        nc[i] = DATA[i].shape[0]
        l = np.round(nc[i] * per).astype(np.uint8)
        if i == 0:
            Labeled = DATA[0][:l, :]
            Unlabeled = DATA[0][l:, :]
        else:
            Labeled = torch.cat((Labeled, DATA[i][:l, :]), 0)
            Unlabeled = torch.cat((Unlabeled, DATA[i][l:, :]), 0)

    u = Unlabeled.shape[0]
    arr = np.arange(u)
    np.random.shuffle(arr)
    Unlabeled = Unlabeled[arr, :]

    data = torch.cat((Labeled, Unlabeled), 0)
    return data


def Alg(A, V, maxiter, num_en, S, DS, D, lam1, lam2):
    H = []
    [n, k, b] = V.shape
    for i in range(num_en):
        H.append(V[:, :, i])
    a = torch.rand(num_en, 1).type(dtype)
    a = a/torch.sum(a)
    h = torch.zeros(num_en, 1).type(dtype)

    eps = torch.tensor(0.00001).type(dtype)
    for it in range(maxiter):
        for kk in range(num_en):
            NUM = A @ H[kk] + lam2 * S @ H[kk]
            DEN = H[kk] @ H[kk].T @ H[kk] + lam2 * DS @ H[kk] + (lam1/2) * D @ H[kk]

            E = (NUM/torch.maximum(DEN, eps))
            H[kk] = H[kk] * (E**(1/4))
            Ht = torch.cdist(H[kk], H[kk], p=2.0)
            h[kk] = torch.norm(A-H[kk] @ H[kk].T)**2 + lam1 * torch.norm(D * (H[kk]@H[kk].T), 1) + lam2 * torch.norm(S * Ht, 1)
            
        sumH=torch.sum(1/h)
        a = (1/h)/sumH

    return H, a

In [ ]:
## GPU or CPU
GPU = False
if GPU:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs", torch.cuda.device_count())
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
    print("CPU")


X, y = torch.load('iris.pt')

n, d = X.shape

data0 = torch.cat((X, y), 1)
p = 0.1
c = torch.unique(y).shape[0]


data = MakeLabel(data0, p, c, n)
X = data[:, :-1]
y = data[:, -1]

K = 8
sigma = 100

A = constructW(X.T, K, sigma)

lam1 = 10
lam2 = 0.001

iters = 10
num_en = 20
maxiter = 500


D, S, DS = sim_dissim(y, n, p)

V = torch.rand(n, c, num_en, iters).type(dtype)
AA = []
HH = []
M = []

for i in range(num_en):
    AA.append(A)
    HH.append(V[:, :, i, 0])
    M.append(torch.zeros([n, c]))
ACC = np.zeros([iters, num_en])
NMI = np.zeros([iters, num_en])

for kk in range(iters):

    [H, a] = Alg(A, V[:, :, :, kk], maxiter, num_en, S, DS, D, lam1, lam2)

    for j in range(num_en):
        M[j] = F.one_hot(torch.argmax(H[j], dim=1), num_classes=c).float()
    sA = torch.zeros(n, n).type(dtype)
    for j in range(num_en):
        sA = sA + a[j] * (M[j] @ M[j].T)
    sA = sA/torch.max(sA)
    print("iteration => ",kk + 1)